In [1]:
import pandas as pd
import numpy as np
import os
import re
import json
import pytz
import datetime
from simple_salesforce import Salesforce, SalesforceLogin, SFType

import presto  # import prestosql python client
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
logininfo = json.load(open('/ghds/groups/labdesk/bshih/salesforce_login.json'))

username = logininfo['username']
password = logininfo['password']
security_token = logininfo['security_token']
domain = 'login'

sf = Salesforce(username=username, password=password, security_token=security_token)

In [3]:
uid = "_svc_acs_presto"
pwd = "em6qF7Gp8E9ECYcKx3gHZ#v$H@pzT7AX3uCJp%5wesKE%VD6gZaCB8*rZ2zNgMKdPkuetSJph2H&H2eadbfCkeB3tBbUqcM^Sj5k"

conn = presto.dbapi.connect(host='acs-exports.bi.ghdna.io',
                            port=8446,
                            user='_svc_acs_presto',
                            catalog='hive',
                            schema='default',
                            http_scheme='https',
                            verify=False,
                            auth=presto.auth.BasicAuthentication(uid, pwd))
cur = conn.cursor()

In [4]:
rbp_srp = []
for r in cur.execute(
        '''SELECT CAST(json_extract(acs_default_cv19.public.operation_operation.input_data, '$.inventory["labware"]["RNA Buffer Plate ID"]') AS VARCHAR),
                  CAST(json_extract(acs_default_cv19.public.operation_operation.input_data, '$.retain_data["retain_plate"]') AS VARCHAR)
           FROM acs_default_cv19.public.operation_operation
           WHERE name = 'Scanning Swab'
        '''):
    rbp_srp.append(r)
    
link = pd.DataFrame(rbp_srp, columns=['RBP', 'SRP']).dropna()

## All Negative Plates

In [5]:
master = pd.read_pickle(
    '/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_call.pickle')

master = master[master['pos_tube_rack'].str.match(r'.*?(A6|B5)') == False]
master = master[master['run_sample_id'].str.startswith(('G', 'H'), na=False)]
master = master[~master['run_sample_id'].str.startswith('Ht', na=False)]

master['RBP'] = master['pos_tube_rack'].str.extract(r'(.*?):.*')
master['Date'] = pd.to_datetime(master['runid'].str[:6], yearfirst=True)
master = master.merge(link, left_on='RBP', right_on='RBP', how='left')

master['total_not_detected'] = master['call'].map({'not_detected': 1,
                                                   'detected': 0,
                                                   'no_call': 0})
master['num_of_samples'] = master['sample_type'].map({'Sample': 1})

srp = master.groupby('RBP', sort=False).agg({'num_of_samples': 'sum',
                                             'total_not_detected': 'sum',
                                             'SRP': 'max',
                                             'Date': 'max'})

srp = srp[(srp['num_of_samples'] == srp['total_not_detected']) & (srp['total_not_detected'] > 70)]\
    .reset_index().sort_values(['Date', 'SRP'], ascending=[False, True])

srp

,RBP,num_of_samples,total_not_detected,SRP,Date
294,RBP201202A203,92,92,SRP200909A439,2021-01-28
293,RBP201202A204,92,92,SRP200909A016,2021-01-27
288,RBP201202A253,92,92,SRP200909A044,2021-01-26
286,RBP201202A250,92,92,SRP200909A045,2021-01-26
292,RBP201202A252,92,92,SRP200909A046,2021-01-26
...,...,...,...,...,...
184,RBP200702A054,92,92,SRP200728A029,2020-08-18
102,RBP200727A040,72,72,SRP200728A084,2020-08-11
101,RBP200727A074,82,82,SRP200728A107,2020-08-11
100,RBP200727A090,153,153,SRP200728A109,2020-08-11


#### SRP with Run Sample ID for PQ

In [6]:
SRP = ['SRP200910A107', 'SRP200910A104', 'SRP200910A210', 'SRP200910A198',
       'SRP200910A204', 'SRP200910A208', 'SRP200910A203', 'SRP200910A133',
       'SRP200901A344', 'SRP200910A170', 'SRP200910A185', 'SRP200910A175']

In [7]:
master[master['SRP'].isin(SRP)].sort_values(by=['SRP', 'RBP']).loc[:, ['SRP', 'run_sample_id']]

,SRP,run_sample_id
56703,SRP200901A344,HBR0723
56704,SRP200901A344,HBT0053
56705,SRP200901A344,HBJ0816
56706,SRP200901A344,HBQ0211
56707,SRP200901A344,HBT0389
...,...,...
69092,SRP200910A210,HBQ0872
69093,SRP200910A210,HBK1148
69094,SRP200910A210,HBT0901
69095,SRP200910A210,HBX0595


# >20K G19 Score Samples Various Sites

In [8]:
high_g19 = master.query("median_covid_ratio > 20000").dropna(axis=1, how='all').sort_values(by='Date', ascending=False).iloc[:70, :]

In [9]:
site_names = []
for i in high_g19['run_sample_id'].unique():
    site_names.extend(sf.query_all(f"SELECT GH_Sample_ID__c, Site_Name__c\
                                     FROM Order WHERE GH_Sample_ID__c = '{i}'").get('records'))

dataframe = pd.DataFrame(site_names)
df = dataframe.drop(columns='attributes').rename(columns={'GH_Sample_ID__c': 'run_sample_id',
                                                          'Site_Name__c': 'Site Name'})

In [10]:
barcode = {}
for i,j in zip(high_g19['run_sample_id'], high_g19['SRP']):
    for r in cur.execute(
            f'''SELECT json_extract(acs_default_cv19.public.operation_operation.input_data, '$.retain_data.sample_retain_map.{i}')
               FROM acs_default_cv19.public.operation_operation
               WHERE CAST(json_extract(acs_default_cv19.public.operation_operation.input_data, '$.retain_data["retain_plate"]') AS VARCHAR) = '{j}'
            '''):
        barcode[i] = json.loads(r[0])

retain_barcode = pd.DataFrame.from_dict(barcode).transpose().reset_index().rename(columns={'index':'run_sample_id'})

In [11]:
sites = high_g19.merge(df, left_on='run_sample_id', right_on='run_sample_id', how='left').merge(retain_barcode, left_on='run_sample_id', right_on='run_sample_id', how='left').sort_values(by='Site Name')
sites[sites['Site Name'] != 'Healing Grove'].sort_values(by=['runid', 'SRP'], ascending=[False, True])

,runid,run_sample_id,sample_type,pos_tube_rack,parameter_set,replicates_count,replicates_detected,replicates_not_detected,replicates_no_call,median_covid_ratio,call,flags,RBP,Date,SRP,total_not_detected,num_of_samples,Site Name,retain_barcode,retain_position
0,210128_NB552482_0011_AHNY2FBGXG,HD20780,Sample,RBP201202A200:D2,Guardant19-RTPCR,3.0,3.0,0.0,0.0,24307.16667,detected,pass,RBP201202A200,2021-01-28,SRP200909A446,0,1,Impossible Foods,4052446217,D2
1,210127_NB552470_0008_AHNY2GBGXG,HDD0016,Sample,RBP201202A050:F7,Guardant19-RTPCR,3.0,3.0,0.0,0.0,38756.50000,detected,pass,RBP201202A050,2021-01-27,SRP200909A041,0,1,Delaware State University,4052463526,F7
21,210115_NB552482_0009_AHJ2VYBGXG,HCB0333,Sample,RBP201202A108:G3,Guardant19-RTPCR,3.0,3.0,0.0,0.0,29601.00000,detected,pass,RBP201202A108,2021-01-15,SRP200909A368,0,1,Delaware State University,4052436942,G3
24,210114_NB552478_0012_AHJ2V7BGXG,HD90487,Sample,RBP201202A218:E10,Guardant19-RTPCR,3.0,3.0,0.0,0.0,30139.00000,detected,pass,RBP201202A218,2021-01-14,SRP200909A021,0,1,QRMD,4052270269,E10
65,210105_NB552393_0054_AHHYTTBGXG,HCG0635,Sample,RBP201112A160:B8,Guardant19-RTPCR,3.0,3.0,0.0,0.0,52799.00000,detected,pass,RBP201112A160,2021-01-05,SRP201026A379,0,1,Lincoln University,4052443223,B8


In [12]:
sites[sites['Site Name'] == 'Healing Grove'].query("SRP != 'SRP200909A025'").sort_values(by=['runid', 'SRP'], ascending=[False, True]).head(39)

,runid,run_sample_id,sample_type,pos_tube_rack,parameter_set,replicates_count,replicates_detected,replicates_not_detected,replicates_no_call,median_covid_ratio,call,flags,RBP,Date,SRP,total_not_detected,num_of_samples,Site Name,retain_barcode,retain_position
2,210126_NB552478_0014_AHNV7JBGXG,HDE0059,Sample,RBP201202A257:E8,Guardant19-RTPCR,3.0,3.0,0.0,0.0,22451.00000,detected,pass,RBP201202A257,2021-01-26,SRP200909A382,0,1,Healing Grove,4052435387,E8
3,210126_NB552478_0014_AHNV7JBGXG,HDE0017,Sample,RBP201202A257:C7,Guardant19-RTPCR,3.0,3.0,0.0,0.0,39462.00000,detected,pass,RBP201202A257,2021-01-26,SRP200909A382,0,1,Healing Grove,4052435362,C7
4,210126_NB552478_0014_AHNV7JBGXG,HDD0686,Sample,RBP201202A255:A3,Guardant19-RTPCR,3.0,2.0,0.0,1.0,23095.39285,detected,pass,RBP201202A255,2021-01-26,SRP200909A383,0,1,Healing Grove,4052435430,A3
10,210121_NB552470_0007_AHNV5VBGXG,HD80065,Sample,RBP201202A029:C8,Guardant19-RTPCR,3.0,3.0,0.0,0.0,23006.66667,detected,pass,RBP201202A029,2021-01-21,SRP200909A422,0,1,Healing Grove,4052469155,C8
7,210121_NB552470_0007_AHNV5VBGXG,HDD0752,Sample,RBP201202A028:B12,Guardant19-RTPCR,3.0,3.0,0.0,0.0,37462.00000,detected,pass,RBP201202A028,2021-01-21,SRP200909A424,0,1,Healing Grove,4052468955,B12
8,210121_NB552470_0007_AHNV5VBGXG,HD90329,Sample,RBP201202A028:H11,Guardant19-RTPCR,3.0,3.0,0.0,0.0,51863.00000,detected,pass,RBP201202A028,2021-01-21,SRP200909A424,0,1,Healing Grove,4052469026,H11
9,210121_NB552470_0007_AHNV5VBGXG,HD90230,Sample,RBP201202A028:B9,Guardant19-RTPCR,3.0,3.0,0.0,0.0,43364.00000,detected,pass,RBP201202A028,2021-01-21,SRP200909A424,0,1,Healing Grove,4052468952,B9
5,210121_NB552470_0007_AHNV5VBGXG,HDA0035,Sample,RBP201202A183:E7,Guardant19-RTPCR,3.0,3.0,0.0,0.0,29202.00000,detected,pass,RBP201202A183,2021-01-21,SRP200909A496,0,1,Healing Grove,4052462074,E7
6,210121_NB552470_0007_AHNV5VBGXG,HD80534,Sample,RBP201202A183:D5,Guardant19-RTPCR,3.0,3.0,0.0,0.0,30176.00000,detected,pass,RBP201202A183,2021-01-21,SRP200909A496,0,1,Healing Grove,4052462060,D5
11,210119_NB552398_0061_AHNTLMBGXG,HD80573,Sample,RBP201202A113:G5,Guardant19-RTPCR,3.0,3.0,0.0,0.0,35143.00000,detected,pass,RBP201202A113,2021-01-19,SRP200909A356,0,1,Healing Grove,4052458832,G5


## >30% Positive

In [13]:
master = pd.read_pickle(
    '/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_call.pickle')
salesforce = pd.read_csv(
    '/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_tat.csv').fillna(0)

master = master[master['pos_tube_rack'].str.match(r'.*?(A6|B5)') == False]
master = master[master['run_sample_id'].str.startswith(('G', 'H'), na=False)]
master = master[~master['run_sample_id'].str.startswith('Ht', na=False)]

master['RBP'] = master['pos_tube_rack'].str.extract(r'(.*?):.*')
master['Date'] = master['runid'].str[:6]

master = master.merge(link, left_on='RBP', right_on='RBP', how='left').dropna(how='all', axis=1).drop_duplicates()

master['total_detected'] = master['call'].map({'not_detected': 0,
                                               'detected': 1,
                                               'no_call': 0})
master['num_of_samples'] = master['sample_type'].map({'Sample': 1})

labels = ['0-0.01', '0.01-1', '1-10', '10-100', '>100']

master['category'] = pd.cut(master['median_covid_ratio'],
                        [-np.inf, 0.01, 1, 10, 100, np.inf], labels=labels)
dummies = pd.get_dummies(master.category)
master = pd.concat([master, dummies], axis=1)

binning = master.groupby('RBP', sort=False).agg({'num_of_samples': 'sum',
                                                 'total_detected': 'sum',
                                                 'SRP': 'max',
                                                 'Date': 'max',
                                                 '0-0.01': 'sum',
                                                 '0.01-1': 'sum',
                                                 '1-10': 'sum',
                                                 '10-100': 'sum',
                                                 '>100': 'sum'})

binning = binning[(binning['total_detected'] > 10) & (binning['Date'] > '2020-10-01')
                  ].reset_index().sort_values(['Date', 'SRP'], ascending=[False, True])

In [14]:
master.iloc[:, 15:].sum()

total_detected     2786
num_of_samples    80534
0-0.01            77748
0.01-1              952
1-10                334
10-100              280
>100               1220
dtype: int64

In [15]:
binning

,RBP,num_of_samples,total_detected,SRP,Date,0-0.01,0.01-1,1-10,10-100,>100
29,RBP201202A224,37,17,SRP200909A053,210128,20,8,2,1,6
28,RBP201202A228,92,13,SRP200909A441,210128,79,6,1,1,5
27,RBP201202A255,92,19,SRP200909A383,210126,73,7,3,2,7
26,RBP201202A256,92,23,SRP200909A388,210126,69,10,4,3,6
25,RBP201202A076,92,15,SRP200909A395,210123,77,6,3,0,6
24,RBP201202A090,92,31,SRP200909A400,210123,61,11,2,3,15
21,RBP201202A028,92,11,SRP200909A424,210121,81,3,0,2,6
23,RBP201202A183,92,19,SRP200909A496,210121,73,8,2,2,7
22,RBP201202A259,92,14,SRP200909A500,210121,78,6,1,3,4
19,RBP201202A113,92,24,SRP200909A356,210119,68,6,4,2,12


In [16]:
RBP = binning.RBP.unique()
binning.sum()['total_detected']

658

In [17]:
only_pos = master[(master['total_detected'] == 1) & (master['Date'] > '2020-10-01') & (master['RBP'].isin(RBP))
                  ].sort_values(by=['run_sample_id', 'Date'], ascending=False)
only_pos = only_pos.merge(
    salesforce, left_on='run_sample_id', right_on='run_sample_id', how='left')
# only_pos.to_csv('retain_plates.csv', index=False)

In [18]:
len(only_pos)

658